In [1]:
import numpy as np
# from scipy.optimize import minimize
# from typing import List, Tuple


In [2]:
def estimate_bt(wins, tol=1e-6, max_iter=1000, prior_strength=0.1):
    """
    Estimate Bradley-Terry beta parameters using an iterative update
    (Minorization-Maximization or fixed-point approach).

    Parameters:
      wins (np.ndarray): a KxK matrix where wins[i, j] counts how many times
                         feature i 'won' over feature j (weighted).
      tol (float): tolerance for convergence.
      max_iter (int): max number of iterations for convergence.
      prior_strength (float): strength of Gaussian(0, σ²=1/prior_strength) prior.
                         0 ⇒ pure MLE; larger ⇒ stronger shrinkage toward 0.

    Returns:
      beta (np.ndarray): array of length K with the estimated BT parameters.
    """
    K = wins.shape[0]
    beta = np.zeros(K)  # Initialize betas at 0
    
    for _ in range(max_iter):
        beta_new = np.copy(beta)
        
        for i in range(K):
            # Sum of all wins for feature i
            wins_i = np.sum(wins[i, :])
            
            # Denominator part of the BT gradient update
            denom = 0.0

            for j in range(K):
                if i == j:
                    continue
                # total comparisons between i and j
                n_ij = wins[i, j] + wins[j, i]
                if n_ij > 0:
                    # add n_ij / (exp(beta[i]) + exp(beta[j]))
                    denom += n_ij / (np.exp(beta[i]) + np.exp(beta[j]))
            
            if wins_i > 0 and denom > 0:
                mle_update = np.log(wins_i) - np.log(denom)
                # MAP update: combine MLE with a zero‐mean Gaussian prior
                beta_new[i] = mle_update / (1.0 + prior_strength)
                # beta_new[i] = np.log(wins_i) - np.log(denom)
        
        if np.max(np.abs(beta_new - beta)) < tol:
            beta = beta_new
            break
        
        beta = beta_new
    
    return beta

def main():
    # 1. Define features and index mapping
    features = ["income", "credit_score", "loan_amount", "employment_type", "education"]
    feature_to_idx = {feat: idx for idx, feat in enumerate(features)}
    K = len(features)
    
    # 2. Initialize wins matrix
    wins = np.zeros((K, K))
    
    # 3. Example survey data
    # Each tuple is (preferred_recourse, not_preferred_recourse).
    actual = 0
    weighted = 0
    survey_data = [
        (["credit_score", "education"],["credit_score", "loan_amount", "education"], 0),
        (["credit_score", "education"],["loan_amount", "education"], 0),
        (["credit_score", "education"],["income", "credit_score", "loan_amount", "education"], 1),
        (["credit_score", "loan_amount", "employment_type"],["education"], 0),
        (["credit_score", "employment_type"],["loan_amount", "education"], 0),
        (["credit_score", "loan_amount", "employment_type"], ["education"], 1),
        (["credit_score"],["loan_amount"], 1),
        (["credit_score", "education"],["loan_amount", "education"], 0),
        (["credit_score"],["loan_amount"], 1),
        (["credit_score"],["credit_score","loan_amount"], 0),
        (["credit_score", "education"],["income", "education"], 0),
        (["credit_score", "loan_amount", "employment_type"],["credit_score", "education"], 1),
        (["credit_score", "education"],["income", "credit_score", "education"], 0),
        (["income", "loan_amount", "employment_type"], ["loan_amount", "education"], 1),
        (["credit_score"],["credit_score","loan_amount"], 1),
        (["credit_score", "employment_type"],["credit_score", "loan_amount", "education"], 1),
        (["credit_score"],["credit_score","loan_amount"], 0),
        (["credit_score", "education"],["loan_amount", "education"], 0),
        (["income", "credit_score", "loan_amount", "employment_type"], ["education"], 1),
        (["credit_score"],["income", "loan_amount"], 0),
        (["credit_score"],["loan_amount"], 1),
        (["credit_score", "loan_amount", "employment_type"], ["credit_score", "education"], 1),
        (["credit_score"],["income", "credit_score"], 0),
        (["credit_score", "education"],["income", "loan_amount", "education"], 0),
        (["income", "credit_score", "loan_amount", "employment_type"], ["credit_score", "education"], 1)
    ]
    
    # 4. Convert each recourse-level preference into feature-level 'wins'
    for recourse_1, recourse_2, pref in survey_data:
        if pref == 0:
            recourse_pref = recourse_1
            recourse_not_pref = recourse_2
            weighted += 1
        else:
            recourse_pref = recourse_2 
            recourse_not_pref = recourse_1
            actual += 1
            
        # Weight each 'win' by 1/(|R_pref|*|R_not_pref|), so each recourse-level
        # preference has the same overall influence, regardless of recourse size.
        weight = 1.0 / (len(recourse_pref) * len(recourse_not_pref))
        
        for feat_pref in recourse_pref:
            for feat_not in recourse_not_pref:
                i = feature_to_idx[feat_pref]
                j = feature_to_idx[feat_not]
                wins[i, j] += weight
    
    # 5. Estimate Bradley-Terry beta parameters
    beta = estimate_bt(wins)
    
   
    # Compute raw costs as the negative of beta
    raw_costs = -beta
    
    # Normalize the costs so that they sum to 1.
    # We exponentiate to ensure all values are positive.
    normalized_costs = np.exp(raw_costs) / np.sum(np.exp(raw_costs))
    
    # Print the results
    print("Feature        Beta     Raw Cost (-Beta)    Normalized Cost (sums to 1)")
    print("-----------------------------------------------------------------------")
    for feat, b, rc, nc in zip(features, beta, raw_costs, normalized_costs):
        print(f"{feat:15s} {b:7.4f} {rc:17.4f} {nc:25.4f}")
    print("-----------------------------------------------------------------------")
    print(f"\nactual: {actual}, weighted: {weighted}")
    print(f"Total: {actual + weighted}")

if __name__ == "__main__":
    main()


Feature        Beta     Raw Cost (-Beta)    Normalized Cost (sums to 1)
-----------------------------------------------------------------------
income          -0.1495            0.1495                    0.6240
credit_score     2.3537           -2.3537                    0.0511
loan_amount      1.7930           -1.7930                    0.0895
employment_type  1.0686           -1.0686                    0.1846
education        2.3576           -2.3576                    0.0509
-----------------------------------------------------------------------

actual: 12, weighted: 13
Total: 25
